In [1]:
from netCDF4 import Dataset
import numpy as np
import xarray as xr
import pandas as pd
import h5py
import os.path
import datetime, calendar
import scipy.io as sio
from pathlib import Path

***Pseudocode:***

1. Extract the DateTime from WRF-Chem and BEHR files
2. Extract the Location from WRF-Chem and BEHR files
3. Align the WRF-Chem and BEHR files by matching DateTime fields
4. Align the WRF-Chem and BEHR files by corresponding geolocations. 

**DateTime extraction from WRF-Chem files**

In [2]:
def WRF_extract_date(filepath): ## grabbing yr, month, day, hr
    filename = os.path.basename(filepath)
    split = filename.split('_')
    filedate = split[2]
    yr, month, day = int(filedate[0:4]), int(filedate[5:7]), int(filedate[8:10])
    hr = int(split[3][0:2])
    return datetime.datetime(yr, month, day, hr)

In [3]:
#test WRF_extract_date method
WRF_filepath = r'wrfout_d01_2013-03-01_02-00-00'
WRF_extract_date(WRF_filepath)

datetime.datetime(2013, 3, 1, 2, 0)

In [4]:
# # DEPRECATED
# def BEHR_extract_date(filepath): ## grabbing yr, month, day - still need hour!
#     filename = os.path.basename(filepath)
#     filedate = filename.split('_')[4]
#     yr, month, day = int(filedate[0:4]), int(filedate[4:6]), int(filedate[6:8])
#     return datetime.datetime(yr, month, day)

In [5]:
# # DEPRECATED
# def BEHR_extract_hour(filepath): #could just add this code to BEHR_extract_date method, but test it here first!
#     behr_contents = sio.loadmat(filepath)
#     behr_data = behr_contents['Data']
#     total_sec = behr_data['Time'] # seconds since Jan1, 1993
    
#     ref_day = datetime.datetime(1993, 1, 1)
#     ref_sec = calendar.timegm(ref_day.timetuple())
#     today = BEHR_extract_date(filepath)
#     today_sec = calendar.timegm(today.timetuple())
    
#      # seconds since midnight = total seconds - (seconds between Jan1, 1993 and our date at midnight)
#     scan_time_sec = total_sec - (today_sec - ref_sec) 
#     scan_time_hr = scan_time_sec / 3600
#     return scan_time_hr

In [6]:
# #test BEHR_extract_date method
# BEHR_filepath = r'OMI_BEHR-DAILY_US_v3-0B_20050103.mat'
# # print(BEHR_extract_date(BEHR_filepath))
# # print(BEHR_extract_hour(BEHR_filepath))

**Preparing OMI/BEHR Files for accessing data variables**

Making a function to extract the OMI files' data from a nested 4x1 array (divided into 4 swaths) into a large n-dimensional array. This will allow one-to-one correlation when using dataframes and make it easier to access data without the swath-nested data structure. 

Using list comprehensions and np.concatenate to optimize runtime. Finding the row axis allows for easy concatenation. 

In [7]:
# extract variables having dimension of any size from nested swaths
def extract_var_anyd(BEHR_filepath, var_name):
    # var_name: String of field name in "Data" array
    # return n-dim array of the data for this n-dim variable (eliminate swaths of nested data)
    behr_contents = sio.loadmat(BEHR_filepath)
    behr_data = behr_contents['Data'][var_name]
#     print(behr_data[0][1].shape)
#     print(len(behr_data[0][1].shape)-2)
    if behr_data[0][1].shape[0] == 1:
        return None
    row_axis = len(behr_data[0][1].shape)-2
    behr_lists = [behr_data[0][s] for s in range(4)] #loop thru all 4 swaths
    flat_data = np.concatenate(behr_lists, row_axis)
#     print(flat_data.shape)
    return flat_data


In [8]:
##### Test extract_var_anyd method 

# extract_var_anyd(BEHR_filepath, 'FoV75Area') # returns None, bc dimensions are 1x60 data
# extract_var_anyd(BEHR_filepath, 'MODISAlbedoFile') # returns None, dim = 1x4
# extract_var_anyd(BEHR_filepath, 'BEHRRegion') # returns None, Text values
# extract_var_anyd(BEHR_filepath, 'Latitude') #2D
# extract_var_anyd(BEHR_filepath, 'CloudFraction') #2D
# extract_var_anyd(BEHR_filepath, 'Time') #1D
# extract_var_anyd(BEHR_filepath, 'FoV75CornerLongitude') #3D


**DateTime extraction from BEHR files**

In [9]:
def BEHR_extract_full_date(filepath):
    # Looks at filename and "Time" variable of the BEHR file
    # Returns an array of datetime.datetime(yr, month, day, hr) instances
    
    # grab total seconds since Jan1, 1993
    total_sec = extract_var_anyd(filepath, 'Time')
    
    # calc seconds since Jan1, 1993
    ref_day = datetime.datetime(1993, 1, 1)
    ref_sec = calendar.timegm(ref_day.timetuple())
#     print(type(ref_sec))
    
    # calc seconds since current day at midnight
    filename = os.path.basename(filepath)
    filedate = filename.split('_')[4]
    yr, month, day = int(filedate[0:4]), int(filedate[4:6]), int(filedate[6:8])
    today = datetime.datetime(yr, month, day)
    today_sec = calendar.timegm(today.timetuple())
#     return datetime.datetime(yr, month, day)

     # seconds since midnight of today = total seconds - (seconds between Jan1, 1993 and our date at midnight)
    scan_time_sec = total_sec - (today_sec - ref_sec) 
    scan_time_hr = scan_time_sec / 3600
    DateTimeArr = np.array([datetime.datetime(yr, month, day, hr) for hr in scan_time_hr])
#     print("Hr array dimension:", scan_time_hr.shape)
#     print("DateTime array dimension:", DateTimeArr.shape)
    return DateTimeArr


In [10]:
### Test BEHR_extract_full_date method

BEHR_filepath = r'OMI_BEHR-DAILY_US_v3-0B_20050103.mat'
BEHR_extract_full_date(BEHR_filepath)

array([datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       datetime.datetime(2005, 1, 3, 17, 0),
       dat

**GeoLocation extraction from files**

In [11]:
## ------------ Opening netCDF Files from WRF-Chem ------------------
# path for the file - "C:\Users\chat2\Documents\Spring/ 2019\BEHR_Proj\wrfout_d01_2013-03-01_02-00-00")
dataset = Path("wrfout_d01_2013-03-01_02-00-00")
file = Dataset(dataset, 'r')
file
# # print(file.variables.keys())
# #listing out all the variables
# for attr in file.ncattrs():
#     print(attr, '=', getattr(file, attr))

print('CEN_LON =', getattr(file, "CEN_LON"))
print('CEN_LAT =', getattr(file, "CEN_LAT"))

CEN_LON = -97.0
CEN_LAT = 38.999996


In [12]:
def WRF_extract_location(filepath): ## grabbing coordinates for WRF-Chem pixel's center
    #returns (lon,lat) for WRF-Chem pixel's center
    dataset = Path(filepath)
    file = Dataset(dataset, 'r')
    return np.array([getattr(file, "CEN_LON"), getattr(file, "CEN_LAT")])

In [13]:
### test WRF_extract_location
wrf_filepath = "wrfout_d01_2013-03-01_02-00-00"
WRF_extract_location(wrf_filepath)

array([-97.      ,  38.999996], dtype=float32)

**Testing for extracting location for BEHR files**

Options: 
    1. Make the "BEHR_extract_location method" to massage the corner lon/lat arrays from BEHR files into 2 4-element arrays
        Feed this into the "interpolate_WRF_to_OMI" in order to collocate BEHR pixels w WRF-Chem pixels
    2. Change the "interpolate_WRF_to_OMI" method to loop thru the BEHR "CornerLon"/"CornerLat" variable arrays. 
        Must be able to index into the Lon/Lat arrays correctly. 
        Will require changing the "interpolate_WRF_to_OMI" method and not just assuming that it works. 

Consider using some numpy array indexing function that can take the corresponding index element from multiple arrays, or will have to make this function myself. 
https://docs.scipy.org/doc/numpy/reference/routines.array-manipulation.html - considering the split functions in particular. 

**Final decision:**
Using "interpolate_WRF_to_OMI" method as helper method for "BEHR_align_location".

"BEHR_align_location" method will, given an OMI and WRF-Chem file, determine if any pixels in the OMI file align w the WRF-Chem file pixel in geolocation.

In [14]:
sum(extract_var_anyd(BEHR_filepath, 'VcdQualityFlags'))


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [15]:
cornerLon = extract_var_anyd(BEHR_filepath, 'FoV75CornerLongitude')
cornerLat = extract_var_anyd(BEHR_filepath, 'FoV75CornerLatitude')

print(cornerLon.shape)
print("# rows: ", cornerLon.shape[1])

rows = cornerLon.shape[1]
for swath in range(4):
    print("swath: ", swath)
    print("size", cornerLon[swath].shape)
#     print(cornerLon[swath])


(4, 795, 60)
# rows:  795
swath:  0
size (795, 60)
swath:  1
size (795, 60)
swath:  2
size (795, 60)
swath:  3
size (795, 60)


In [16]:
# should print out 4-corner arrays
rows = cornerLon.shape[1]
cols = cornerLon.shape[2]
for i in range(rows):
    for j in range(cols):
        print(cornerLon[:, i, j])


[-76.50710297 -75.10810089 -75.18730164 -76.59929657]
[-75.10810089 -73.98329926 -74.05470276 -75.18730164]
[-73.98329926 -73.04519653 -73.11049652 -74.05470276]
[-73.04519653 -72.24140167 -72.30190277 -73.11049652]
[-72.24140167 -71.53919983 -71.59580231 -72.30190277]
[-71.53919983 -70.91570282 -70.96900177 -71.59580231]
[-70.91570282 -70.35540009 -70.40579987 -70.96900177]
[-70.35540009 -69.84600067 -69.89389801 -70.40579987]
[-69.84600067 -69.3789978  -69.42479706 -69.89389801]
[-69.3789978  -68.94709778 -68.9910965  -69.42479706]
[-68.94709778 -68.54530334 -68.58769989 -68.9910965 ]
[-68.54530334 -68.16880035 -68.20970154 -68.58769989]
[-68.16880035 -67.81420135 -67.85389709 -68.20970154]
[-67.81420135 -67.478302   -67.51689911 -67.85389709]
[-67.478302   -67.1588974  -67.19650269 -67.51689911]
[-67.1588974  -66.85350037 -66.89029694 -67.19650269]
[-66.85350037 -66.56069946 -66.59680176 -66.89029694]
[-66.56069946 -66.2786026  -66.31390381 -66.59680176]
[-66.2786026  -66.0059967  -

[-63.99750137 -63.76509857 -63.79639816 -64.02899933]
[-63.76509857 -63.53250122 -63.56359863 -63.79639816]
[-63.53250122 -63.29909897 -63.33000183 -63.56359863]
[-63.29909897 -63.06439972 -63.09500122 -63.33000183]
[-63.06439972 -62.82740021 -62.85789871 -63.09500122]
[-62.82740021 -62.58779907 -62.61800003 -62.85789871]
[-62.58779907 -62.34469986 -62.37469864 -62.61800003]
[-62.34469986 -62.09730148 -62.12720108 -62.37469864]
[-62.09730148 -61.84489822 -61.87469864 -62.12720108]
[-61.84489822 -61.58670044 -61.61629868 -61.87469864]
[-61.58670044 -61.32149887 -61.35110092 -61.61629868]
[-61.32149887 -61.04850006 -61.07799911 -61.35110092]
[-61.04850006 -60.76639938 -60.79579926 -61.07799911]
[-60.76639938 -60.47389984 -60.50329971 -60.79579926]
[-60.47389984 -60.16949844 -60.19889832 -60.50329971]
[-60.16949844 -59.85160065 -59.88100052 -60.19889832]
[-59.85160065 -59.51800156 -59.54750061 -59.88100052]
[-59.51800156 -59.16680145 -59.19630051 -59.54750061]
[-59.16680145 -58.79489899 -

[-67.51979828 -67.22979736 -67.26719666 -67.55799866]
[-67.22979736 -66.94960022 -66.98629761 -67.26719666]
[-66.94960022 -66.67759705 -66.71369934 -66.98629761]
[-66.67759705 -66.41300201 -66.44860077 -66.71369934]
[-66.41300201 -66.15460205 -66.18969727 -66.44860077]
[-66.15460205 -65.90170288 -65.93630219 -66.18969727]
[-65.90170288 -65.65319824 -65.68740082 -65.93630219]
[-65.65319824 -65.40869904 -65.44239807 -65.68740082]
[-65.40869904 -65.16699982 -65.20040131 -65.44239807]
[-65.16699982 -64.92780304 -64.96080017 -65.20040131]
[-64.92780304 -64.69020081 -64.72290039 -64.96080017]
[-64.69020081 -64.4536972  -64.48609924 -64.72290039]
[-64.4536972  -64.21759796 -64.24970245 -64.48609924]
[-64.21759796 -63.98130035 -64.01309967 -64.24970245]
[-63.98130035 -63.74420166 -63.77569962 -64.01309967]
[-63.74420166 -63.50559998 -63.53689957 -63.77569962]
[-63.50559998 -63.26480103 -63.29579926 -63.53689957]
[-63.26480103 -63.02130127 -63.05210114 -63.29579926]
[-63.02130127 -62.77410126 -

[-55.01869965 -54.13819885 -54.16949844 -55.04999924]
[-54.13819885 -53.11050034 -53.14189911 -54.16949844]
[-53.11050034 -51.88079834 -51.9117012  -53.14189911]
[-51.88079834 -50.35469818 -50.38560104 -51.9117012 ]
[-78.25679779 -76.81880188 -76.90769958 -78.36039734]
[-76.81880188 -75.66069794 -75.74040222 -76.90769958]
[-75.66069794 -74.69309998 -74.76589966 -75.74040222]
[-74.69309998 -73.86309814 -73.93039703 -74.76589966]
[-73.86309814 -73.13700104 -73.19979858 -73.93039703]
[-73.13700104 -72.49189758 -72.55090332 -73.19979858]
[-72.49189758 -71.91139984 -71.96710205 -72.55090332]
[-71.91139984 -71.38349915 -71.43640137 -71.96710205]
[-71.38349915 -70.89900208 -70.94940186 -71.43640137]
[-70.89900208 -70.45079803 -70.49909973 -70.94940186]
[-70.45079803 -70.03330231 -70.07990265 -70.49909973]
[-70.03330231 -69.64209747 -69.68689728 -70.07990265]
[-69.64209747 -69.27330017 -69.31680298 -69.68689728]
[-69.27330017 -68.92389679 -68.9661026  -69.31680298]
[-68.92389679 -68.59140015 -

[-60.36470032 -59.99959946 -60.0298996  -60.39500046]
[-59.99959946 -59.61309814 -59.64350128 -60.0298996 ]
[-59.61309814 -59.20199966 -59.23239899 -59.64350128]
[-59.20199966 -58.76210022 -58.79259872 -59.23239899]
[-58.76210022 -58.2887001  -58.31940079 -58.79259872]
[-58.2887001  -57.77560043 -57.8064003  -58.31940079]
[-57.77560043 -57.21530151 -57.24620056 -57.8064003 ]
[-57.21530151 -56.59780121 -56.62889862 -57.24620056]
[-56.59780121 -55.91030121 -55.94150162 -56.62889862]
[-55.91030121 -55.13510132 -55.16650009 -55.94150162]
[-55.13510132 -54.2478981  -54.27930069 -55.16650009]
[-54.2478981  -53.21260071 -53.24399948 -54.27930069]
[-53.21260071 -51.97370148 -52.00460052 -53.24399948]
[-51.97370148 -50.43640137 -50.46699905 -52.00460052]
[-78.52480316 -77.08039856 -77.17089844 -78.63040161]
[-77.08039856 -75.91670227 -75.99780273 -77.17089844]
[-75.91670227 -74.94419861 -75.01830292 -75.99780273]
[-74.94419861 -74.10990143 -74.17829895 -75.01830292]
[-74.10990143 -73.37979889 -

...

[-124.10800171 -123.75700378 -123.82099915 -124.1740036 ]
[-123.75700378 -123.40899658 -123.47200012 -123.82099915]
[-123.40899658 -123.06400299 -123.12599945 -123.47200012]
[-123.06400299 -122.72000122 -122.78099823 -123.12599945]
[-122.72000122 -122.3769989  -122.43699646 -122.78099823]
[-122.3769989  -122.03399658 -122.09300232 -122.43699646]
[-122.03399658 -121.69000244 -121.7480011  -122.09300232]
[-121.69000244 -121.34400177 -121.40100098 -121.7480011 ]
[-121.34400177 -120.9960022  -121.05200195 -121.40100098]
[-120.9960022  -120.64299774 -120.69799805 -121.05200195]
[-120.64299774 -120.28600311 -120.33999634 -120.69799805]
[-120.28600311 -119.92299652 -119.97599792 -120.33999634]
[-119.92299652 -119.5530014  -119.60500336 -119.97599792]
[-119.5530014  -119.1740036  -119.22599792 -119.60500336]
[-119.1740036  -118.78600311 -118.83699799 -119.22599792]
[-118.78600311 -118.38800049 -118.43800354 -118.83699799]
[-118.38800049 -117.97699738 -118.02600098 -118.43800354]
[-117.97699738

[-117.45600128 -116.98500061 -117.03299713 -117.50499725]
[-116.98500061 -116.49199677 -116.53900146 -117.03299713]
[-116.49199677 -115.97499847 -116.02100372 -116.53900146]
[-115.97499847 -115.42900085 -115.47399902 -116.02100372]
[-115.42900085 -114.84999847 -114.89499664 -115.47399902]
[-114.84999847 -114.23500061 -114.27799988 -114.89499664]
[-114.23500061 -113.5759964  -113.61699677 -114.27799988]
[-113.5759964  -112.86599731 -112.90599823 -113.61699677]
[-112.86599731 -112.09600067 -112.13400269 -112.90599823]
[-112.09600067 -111.2559967  -111.29199982 -112.13400269]
[-111.2559967  -110.33000183 -110.36299896 -111.29199982]
[-110.33000183 -109.2990036  -109.32900238 -110.36299896]
[-109.2990036  -108.13800049 -108.16400146 -109.32900238]
[-108.13800049 -106.81099701 -106.83200073 -108.16400146]
[-106.81099701 -105.26499939 -105.27899933 -106.83200073]
[-105.26499939 -103.42199707 -103.4260025  -105.27899933]
[-103.42199707 -101.14499664 -101.13600159 -103.4260025 ]
[-140.91700745

[-137.17100525 -136.08500671 -136.23300171 -137.33200073]
[-136.08500671 -135.1230011  -135.26100159 -136.23300171]
[-135.1230011  -134.25999451 -134.38900757 -135.26100159]
[-134.25999451 -133.4750061  -133.59700012 -134.38900757]
[-133.4750061  -132.7559967  -132.8710022  -133.59700012]
[-132.7559967  -132.09100342 -132.19999695 -132.8710022 ]
[-132.09100342 -131.47000122 -131.57499695 -132.19999695]
[-131.47000122 -130.88900757 -130.99000549 -131.57499695]
[-130.88900757 -130.34100342 -130.43699646 -130.99000549]
[-130.34100342 -129.82099915 -129.91400146 -130.43699646]
[-129.82099915 -129.32600403 -129.41600037 -129.91400146]
[-129.32600403 -128.852005   -128.94000244 -129.41600037]
[-128.852005   -128.39700317 -128.48199463 -128.94000244]
[-128.39700317 -127.95899963 -128.04100037 -128.48199463]
[-127.95899963 -127.53399658 -127.61499786 -128.04100037]
[-127.53399658 -127.12200165 -127.2009964  -127.61499786]
[-127.12200165 -126.72100067 -126.79799652 -127.2009964 ]
[-126.72100067

[-139.58299255 -139.33599854 -139.3690033  -139.61700439]
[-139.33599854 -139.09300232 -139.12600708 -139.3690033 ]
[-139.09300232 -138.85400391 -138.88699341 -139.12600708]
[-138.85400391 -138.61799622 -138.6499939  -138.88699341]
[-138.61799622 -138.38499451 -138.41700745 -138.6499939 ]
[-138.38499451 -138.15299988 -138.18400574 -138.41700745]
[-138.15299988 -137.92199707 -137.95300293 -138.18400574]
[-137.92199707 -137.69099426 -137.72200012 -137.95300293]
[-137.69099426 -137.46099854 -137.49200439 -137.72200012]
[-137.46099854 -137.22900391 -137.25999451 -137.49200439]
[-137.22900391 -136.99699402 -137.0269928  -137.25999451]
[-136.99699402 -136.76199341 -136.79200745 -137.0269928 ]
[-136.76199341 -136.52400208 -136.55400085 -136.79200745]
[-136.52400208 -136.28300476 -136.31300354 -136.55400085]
[-136.28300476 -136.03799438 -136.06700134 -136.31300354]
[-136.03799438 -135.78700256 -135.81700134 -136.06700134]
[-135.78700256 -135.53100586 -135.56100464 -135.81700134]
[-135.53100586

[-134.93099976 -134.63800049 -134.66799927 -134.96000671]
[-134.63800049 -134.33399963 -134.36399841 -134.66799927]
[-134.33399963 -134.01600647 -134.04600525 -134.36399841]
[-134.01600647 -133.68299866 -133.71299744 -134.04600525]
[-133.68299866 -133.33200073 -133.36099243 -133.71299744]
[-133.33200073 -132.96000671 -132.99000549 -133.36099243]
[-132.96000671 -132.56500244 -132.59500122 -132.99000549]
[-132.56500244 -132.14199829 -132.17199707 -132.59500122]
[-132.14199829 -131.68699646 -131.71699524 -132.17199707]
[-131.68699646 -131.19299316 -131.22399902 -131.71699524]
[-131.19299316 -130.65400696 -130.68499756 -131.22399902]
[-130.65400696 -130.06100464 -130.09199524 -130.68499756]
[-130.06100464 -129.39900208 -129.43099976 -130.09199524]
[-129.39900208 -128.65400696 -128.68600464 -129.43099976]
[-128.65400696 -127.8010025  -127.83300018 -128.68600464]
[-127.8010025  -126.80500031 -126.83799744 -127.83300018]
[-126.80500031 -125.61299896 -125.64600372 -126.83799744]
[-125.61299896

In [17]:
def interpolate_WRF_to_OMI(wrf_pixel_arr, omi_lons, omi_lats): 
    # returns true if wrf_pixel belongs to this OMI pixel, OR can set OMI pixel attribute to true
    # checks that wrf_pixel's center lies within the box formed by the OMI pixel's corners
    # input: wrf_pixel_arr - 2-element array - longitude and latitude of wrf pixel's center
    #        omi_pixel_arr - array containing 2 4-elt arrays (longitude and latitude arrays of OMI pixel's 4 corners)
    # (if going thru all the pixels in an OMI file, return an array of booleans)

    wrf_lon, wrf_lat = wrf_pixel_arr[0], wrf_pixel_arr[1]
    min_lon, max_lon = min(omi_lons), max(omi_lons)
    min_lat, max_lat = min(omi_lats), max(omi_lats)
    
    return (wrf_lon > min_lon and wrf_lon < max_lon and 
            wrf_lat > min_lat and wrf_lon < max_lat)

In [18]:
# Testing for interpolate_WRF_to_OMI method
# assume A is a wrf pixel, O is a OMI pixel
W = [0, 0, 12]
O_lon = [-1, 1, 1, -1]
O_lat = [1, 1, -1, -1]
print(interpolate_WRF_to_OMI(W, O_lon, O_lat)) #should return True

W = [0, 0, 12]
O_lon = [-1, 1, 2, 0]
O_lat = [1, 1, -1, -1]
print(interpolate_WRF_to_OMI(W, O_lon, O_lat)) #should return True

W = [0, 0, 12]
O_lon = [-1, 1, 1, -1]
O_lat = [1, 1, 0, 0]
print(interpolate_WRF_to_OMI(W, O_lon, O_lat)) #should return False

True
True
False


In [19]:
# # Testing that looping through coordinates works properly

# W = [0, 0, 12]
# rows = cornerLon.shape[1]
# cols = cornerLon.shape[2]
# for i in range(rows):
#     for j in range(cols):
#         print("entry: " ,i , ", " , j) 
#         O_lon = cornerLon[:, i, j]
#         O_lat = cornerLat[:, i, j]
#         print("lon: ", O_lon)
#         print("lat: ", O_lat)
#         print(interpolate_WRF_to_OMI(W, O_lon, O_lat))

In [20]:
def BEHR_align_location(wrf_file, Behr_file): ## grabbing coordinates for OMI pixel's Corner Lon/Lat arrays 
    
    # for each BEHR pixel, we use 2 4-elt arrays (longitude and latitude arrays of OMI pixel's 4 corners)
    # we use interpolate_WRF_to_OMI method to check if given WRF-center and OMI-corners,
    # whether or not the WRF-pixel belongs to the OMI pixel
    # we have 795*60 OMI pixels in total to check
    # return a numpy 795*60 boolean array  
    
    W = WRF_extract_location(wrf_file)
    cornerLon = extract_var_anyd(Behr_file, 'FoV75CornerLongitude')
    cornerLat = extract_var_anyd(Behr_file, 'FoV75CornerLatitude')
    
    rows = cornerLon.shape[1]
    cols = cornerLon.shape[2]
    bool_arr = np.full((795,60), True)
    for i in range(rows):
        for j in range(cols):
            O_lon = cornerLon[:, i, j]
            O_lat = cornerLat[:, i, j]
            bool_arr[i][j] = interpolate_WRF_to_OMI(W, O_lon, O_lat)

#     bool_df = pandas.DataFrame(bool_arr)
        
    return bool_arr

In [21]:
### test BEHR_align_location
wrf_filepath = "wrfout_d01_2013-03-01_02-00-00"
BEHR_filepath = r'OMI_BEHR-DAILY_US_v3-0B_20050103.mat'
BEHR_align_location(wrf_filepath, BEHR_filepath)


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

buffer space for testing

In [23]:
# def BEHR_extract_var_to_df(filepath, variable_list):
#     # use this function to extract certain BEHR variables and return a dataframe containing these var
        

In [24]:
# # test BEHR_extract_var_to_df method
# # for BEHR files, we want the corner coordinates and time stamp
# var_list = ['Longitude', 'Latitude', '']
# BEHR_extract_var_to_df(BEHR_filepath, )

In [25]:
# TESTINGGGGGGGGG
behr_contents = sio.loadmat(BEHR_filepath)
print(type(behr_contents))
print(sorted(behr_contents.keys())) 
print(behr_contents['Data']['Time'])

<class 'dict'>
['Data', 'OMI', '__function_workspace__', '__globals__', '__header__', '__version__']
[[array([[3.78927566e+08],
       [3.78927568e+08],
       [3.78927570e+08],
       [3.78927572e+08],
       [3.78927574e+08],
       [3.78927576e+08],
       [3.78927578e+08],
       [3.78927580e+08],
       [3.78927582e+08],
       [3.78927584e+08],
       [3.78927586e+08],
       [3.78927588e+08],
       [3.78927590e+08],
       [3.78927592e+08],
       [3.78927594e+08],
       [3.78927596e+08],
       [3.78927598e+08],
       [3.78927600e+08],
       [3.78927602e+08],
       [3.78927604e+08],
       [3.78927606e+08],
       [3.78927608e+08],
       [3.78927610e+08],
       [3.78927612e+08],
       [3.78927614e+08],
       [3.78927616e+08],
       [3.78927618e+08],
       [3.78927620e+08],
       [3.78927622e+08],
       [3.78927624e+08],
       [3.78927626e+08],
       [3.78927628e+08],
       [3.78927630e+08],
       [3.78927632e+08],
       [3.78927634e+08],
       [3.78927636e+08

**Aligning Geolocations**

In [27]:
W = [0, 0, 12]
rows = cornerLon.shape[1]
cols = cornerLon.shape[2]
bool_arr = np.full((795,60), True)
for i in range(rows):
    for j in range(cols):
        O_lon = cornerLon[:, i, j]
        O_lat = cornerLat[:, i, j]
        bool_arr[i][j] = interpolate_WRF_to_OMI(W, O_lon, O_lat)

bool_df = pd.DataFrame(bool_arr)

In [28]:
bool_df

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [29]:
bool_df.count_values()

AttributeError: 'DataFrame' object has no attribute 'count_values'